In [1]:
#file as of 12/18/2019

import pandas as pd
from scipy.io import wavfile
from scipy import signal
import os
import matplotlib.pyplot as plt
import time
import numpy as np
from PIL import Image
%pylab
%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Note: I've moved to using 1s window slices

### Define where the raw data files are located
I've separated the training data from the test data.

In [30]:
indir_wav = 'E:/CJunk/Turkey_gobble_identification/Darren_Data/test/'
indir_data = 'E:/CJunk/Turkey_gobble_identification/Darren_Data/test/'
outdir = 'E:/CJunk/Turkey_gobble_identification/turkey_classifier_short_window/test/

In [34]:
indir_wav = 'E:/CJunk/Turkey_gobble_identification/Darren_Data/train/'
indir_data = 'E:/CJunk/Turkey_gobble_identification/Darren_Data/train/'
outdir = 'E:/CJunk/Turkey_gobble_identification/turkey_classifier_short_window/temp/

### Identify the data in the folders
Gets a list of the data files for running through with the code below

In [35]:
allfiles = os.listdir(indir_wav)
files = []
for x in range(len(allfiles)):
    if allfiles[x][-3:] == 'wav':
        files.append(allfiles[x][:-4])
files = list(set(files))

infilename = files[-2]
print(files)
print(len(files))
print(infilename)

['CC20_20180414_080002', 'CC20_20180414_100002', 'CC16_20180303_090002', 'CC32_20180315_060002', 'CC16_20180303_100002', 'CC32_20180406_070002', 'CC32_20180413_060002', 'CC32_20180317_060002', 'CC32_20180409_090002', 'CC32_20180318_070002', 'CC32_20180312_070002', 'CC32_20180316_060002', 'CC20_20180414_090002', 'CC16_20180303_070002', 'CC32_20180314_070002', 'CC32_20180317_080002', 'CC32_20180314_060002', 'CC32_20180524_080002', 'CC32_20180423_080002', 'CC20_20180414_070002', 'CC20_20180414_060002', 'CC16_20180303_080002', 'CC32_20180301_080002', 'CC32_20180315_070002']
24
CC32_20180301_080002


### This section will read the TSV files and locate the events in them
This outputs 'high resolution' data from the here with window shifts of 300 (~0.0125s shifts). Since we don't always know where the gobble will be located with respect to the window when classifying new data, we want to have an example of the gobble in approximately every possible location within the window.

In [36]:
#this one is to use the TSV files of validated data and focus only on these regions
for infile in files[:]:
    infilename = infile
    print('File: ', infile)
    wav = wavfile.read(indir_wav + infilename + '.wav')
    dat = pd.read_csv(indir_data + infilename + '.txt', sep='\t', header=0)

    dat['mod_begin'] = round(dat['Begin Time (s)']*24000).astype('int')
    dat['mod_end'] = round(dat['End Time (s)']*24000).astype('int')
    if 'Gobbles' in dat.columns:
        dat['Gobble'] = dat['Gobbles']

    for x in range(len(dat)):
        time1 = time.time()
        temp_data = wav[1]
        begin_data = dat.iloc[x]['mod_begin']
        gobble = dat.iloc[x]['Gobble']
        start_ind = dat.iloc[x]['mod_begin']
        end_ind = dat.iloc[x]['mod_end']
        check_for_gobble = False
        if gobble == 0: #need to evaluate whether the sliding window will encompass an actual gobble
                        #if it does, then I'll have to code as a gobble
            if 0 < x < len(dat) - 1:
                if dat.iloc[x-1]['mod_end'] > start_ind - 6000 and dat.iloc[x-1]['Gobble'] == 1:
                    check_for_gobble = True
                elif dat.iloc[x+1]['mod_begin'] < end_ind + 6000 and dat.iloc[x+1]['Gobble'] == 1:
                    check_for_gobble = True
            if 1 < x < len(dat) - 2:
                if dat.iloc[x-2]['mod_end'] > start_ind - 6000 and dat.iloc[x-2]['Gobble'] == 1:
                    check_for_gobble = True
                elif dat.iloc[x+2]['mod_begin'] < end_ind + 6000 and dat.iloc[x+2]['Gobble'] == 1:
                    check_for_gobble = True
                    
        for y in range(1000,24001,300):
            #try:#signal in first part
                f, t, Zxx = signal.stft(temp_data[begin_data+y:begin_data+24000+y], 24000, nperseg=1000)
                a = Image.fromarray(np.abs(Zxx)[25:150].astype('int16')*10)
                a = a.convert('L')
                if check_for_gobble:
                    #if dat.iloc[x-1]['mod_end'] - 6000 > (begin_data-y) and dat.iloc[x-1]['Gobble'] == 1:
                    #    gobble = 1
                    #elif dat.iloc[x+1]['mod_begin'] + 6000 < (begin_data+24000-y) and dat.iloc[x+1]['Gobble'] == 1:
                    #    gobble = 1
                    if dat.iloc[x-1]['mod_end'] - (begin_data-y) > 6000 and dat.iloc[x-1]['Gobble'] == 1:
                        gobble = 1
                    elif (begin_data+24000-y) - dat.iloc[x+1]['mod_begin'] > 6000  and dat.iloc[x+1]['Gobble'] == 1:
                        gobble = 1
                        
                if gobble == 1:
                    a.save(outdir + 'gobble/' + infilename + '_call_'+str(x) + '_' + str(y) +'.png')
                else:
                    a.save(outdir +'other/' + infilename + '_call_'+str(x) + '_' + str(y) +'.png')
                if check_for_gobble:
                    gobble = 0
                
            #except:
            #    pass
        plt.close()

File:  CC20_20180414_080002
File:  CC20_20180414_100002
File:  CC16_20180303_090002
File:  CC32_20180315_060002
File:  CC16_20180303_100002
File:  CC32_20180406_070002
File:  CC32_20180413_060002
File:  CC32_20180317_060002
File:  CC32_20180409_090002
File:  CC32_20180318_070002
File:  CC32_20180312_070002
File:  CC32_20180316_060002
File:  CC20_20180414_090002
File:  CC16_20180303_070002
File:  CC32_20180314_070002
File:  CC32_20180317_080002
File:  CC32_20180314_060002
File:  CC32_20180524_080002
File:  CC32_20180423_080002
File:  CC20_20180414_070002
File:  CC20_20180414_060002
File:  CC16_20180303_080002
File:  CC32_20180301_080002
File:  CC32_20180315_070002


### Run through an entire WAV file
This section will slice an entire WAV file into windows (sliding 0.25s between windows), check to see if the window overlaps a known gobble (from the TSV of validated data) and outputs it to the proper folder. This section helps to grab region of time where no events were detected (more cases of negative examples). <br> Since there are missed gobbles in the files, these will be put into the 'other' category. Running a training file through the CNN can highlight these missed gobbles and, after being validated, they can be added to the TSV file so they're picked up as gobbles whenever the training set is regenerated. I have been training the model, running it on the training data, finding missed gobbles, adding them in, regenerating the training set images, and then training the model more with this 'better' dataset. 


In [37]:
#this block is for creating the windows based on 1s blocks of time (NOT knowing where the calls are)
#this is for the training data; it seems I missed quite a bit of the 'silence' in the files

for infile in files[:]:
    infilename = infile
    print('File: ', infile)
    wav = wavfile.read(indir_wav + infilename + '.wav')
    dat = pd.read_csv(indir_data + infilename + '.txt', sep='\t', header=0)

    dat['mod_begin'] = round(dat['Begin Time (s)']*24000).astype('int')
    dat['mod_end'] = round(dat['End Time (s)']*24000).astype('int')
    if 'Gobbles' in dat.columns:
        dat['Gobble'] = dat['Gobbles']
    current_event = 0
    temp_data = wav[1]
    #if sum(dat['Gobble']) < 1:
    #    continue
    for x in range(0, len(wav[1])+1, 6000):
        begin_data = x
        start_ind = x
        end_ind = x + 24000
        if end_ind >= len(wav[1]):
            continue
        
        check_for_gobble = False
        gobble = 0
        if current_event < len(dat) - 1 and dat.iloc[current_event]['Gobble'] == 1:
            if dat.iloc[current_event]['mod_end'] - start_ind > 6000 and end_ind - dat.iloc[current_event]['mod_begin'] > 6000:
                check_for_gobble = True
                gobble = 1
            
        
        while begin_data > dat.iloc[current_event]['mod_end']:
            current_event += 1
            if current_event == len(dat):
                current_event -= 1
                break
        #if check_for_gobble:
        f, t, Zxx = signal.stft(temp_data[begin_data:end_ind], 24000, nperseg=1000, noverlap=500)
        a = Image.fromarray(np.abs(Zxx)[25:150].astype('int16'))
        a = a.convert('L')
        if gobble == 1:
            #pass
            a.save(outdir + 'gobble/' + infilename + '_call_{:08d}.png'.format(x))
        else:
            a.save(outdir + 'other/' + infilename + '_call_{:08d}.png'.format(x))


File:  CC20_20180414_080002
File:  CC20_20180414_100002
File:  CC16_20180303_090002
File:  CC32_20180315_060002
File:  CC16_20180303_100002
File:  CC32_20180406_070002
File:  CC32_20180413_060002
File:  CC32_20180317_060002
File:  CC32_20180409_090002
File:  CC32_20180318_070002
File:  CC32_20180312_070002
File:  CC32_20180316_060002
File:  CC20_20180414_090002
File:  CC16_20180303_070002
File:  CC32_20180314_070002
File:  CC32_20180317_080002
File:  CC32_20180314_060002
File:  CC32_20180524_080002
File:  CC32_20180423_080002
File:  CC20_20180414_070002
File:  CC20_20180414_060002
File:  CC16_20180303_080002
File:  CC32_20180301_080002
File:  CC32_20180315_070002
